In [1]:
from itertools import product
from math import exp
import numpy as np
from numba import njit, prange

In [2]:
@njit
def int2array(x: int, L: int) -> np.ndarray:
    res = np.empty(L, dtype=np.int8)
    for i in range(L):
        res[i] = (x & 1) * 2 - 1
        x = x >> 1
    return res

In [3]:
def get_range(a : int, b : int) -> np.array:
    '''
    get_range takes a and b and return range(a, b, 1). The difference is that this one return list.
    '''
    res = []
    while a <= b:
        res.append(a)
        a += 1
    return res

In [4]:

class find_mean_energy_for_1d:
    def int2array(self, x: int, L: int) -> np.ndarray:
        '''
        Transform integer x to bit represantation.
        '''
        res = np.empty(L, dtype=np.int8)
        for i in range(L):
            res[i] = (x & 1) * 2 - 1
            x = x >> 1
        return res
    def energy(self, sigma: np.ndarray) -> int:
        '''
        Return the energy of set of electrons when the spin orientation accords to sigma. It is working for 1D case.
        '''
        E = 0
        n = len(sigma)
        for i in range(n):
            E -= sigma[i] * sigma[(i + 1) % n]
        return E
    def mean_energy(self, L: int, kT: np.array) -> float:
        '''
        Return the mean energy of set of electrons with spin orientation accords to sigma and temperature equals to kT.
        '''
        E_mean = 0
        Z = 0
        for sigma in prange(2**(L-1)):
            E = self.energy(int2array(sigma, L))
            e = exp(-E / kT)
            E_mean += E * e
            Z += e
        E_mean /= Z
        return E_mean / L
    
@njit
def energy_for_2d(sigma) -> int:
    '''
    energy_for_2d calculates energy for given 2D matrix which contains information about spin orientation of electrons.
    '''
    E = 0
    n = len(sigma)
    m = len(sigma[0])
    for i in prange(-1, n - 1):
        for j in prange(-1, m - 1):
            E -= (sigma[i][j] * sigma[(i + 1)][j] + sigma[i][j] * sigma[i][j + 1])
    return E


In [5]:
@njit
def calculate_mean_energy_for_2d(sigma : np.array, Lx : int, E_mean : float, Z : float, kT : float) -> np.array:
    '''
    calculate_mean_energy_for_2d takes np.array numbers_for_translation_to_two_bits_base each number of which will be used to transfer bite representation
    to all spin sets then will use sigma to determine the mean energy E_mean. 
    '''
    E = energy_for_2d(sigma)
    e = np.exp(-E / kT)
    Z += e
    E_mean += E * e
    return np.array([E_mean, Z])

def mean_energy(Lx: int, Ly: int, kT: np.array) -> np.array:
    '''
    mean_energy function contains logic for finding mean energy for 1D case and 2D case. It depends on Lx and Ly which logic it will choose.
    '''
    i = 0
    if Lx == 1:
        res = np.empty(Ly)
        for current_kT in kT:
            res[i] = find_mean_energy_for_1d().mean_energy(Ly, current_kT)
            i += 1
        return res
    elif Ly == 1:
        res = np.empty(Lx)
        for current_kT in kT:
            res[i] = find_mean_energy_for_1d().mean_energy(Lx, current_kT)
            i += 1
        return res
    else:
        return mean_energy_for_2d(Lx, Ly, kT)
@njit(parallel=True)
def mean_energy_for_2d(Lx: int, Ly : int, kT: np.array) -> np.array:
    '''
    mean_energy_for_2d calcultate mean energy of spin sets with Lx * Ly electrons. Lx number of rows and Ly number of columns in this matrix.
    '''

    N = Lx * Ly
    unscaled_energies = np.empty(2**N)
    probs = np.empty(2**N)
    res = np.empty(len(kT))
    for current_kT in prange(len(kT)):
            i = 0 
            E_mean = 0
            Z = 0
            for k in prange(2**N):
                sigma = np.empty(N, dtype=np.int8)
                flag = np.int64(k)
                for index in range(N):
                    if flag & 1 == 0:
                        sigma[index] = -1
                    else:
                        sigma[index] = 1
                    flag = flag >> 1
                sigma = sigma.reshape(Ly, Lx)
                current_res = calculate_mean_energy_for_2d(sigma, Lx, E_mean, Z, kT[current_kT])
                E_mean = current_res[0]
                Z = current_res[1]
            E_mean /= Z
            res[i] = E_mean / (Lx * Ly)
            i += 1

        
    return res

In [ ]:
import time
from tqdm import trange, tqdm
# Initialization of data. Put your input here.
kT_range = np.arange(1.0, 5.0, 0.1) # Before multiplying by 0.1 and adding 1.0
Ly = 4
Lx_range = np.arange(2, 9, 1)

#Main logic
first_element_of_Lx_range = Lx_range[0]
array_of_mean_energies = np.empty(shape=(Lx_range[-1] - Lx_range[0], int(kT_range[-1] * 10) - int(kT_range[0] * 10) + 1))
print(kT_range[-1])
for Lx in trange(2, 9):
    j = 0
    array_of_mean_energies[Lx - first_element_of_Lx_range] = mean_energy(Lx, Ly, kT_range)
np.save("matrix_of_mean_energies.txt", array_of_mean_energies)

4.900000000000004


 86%|███████████████████████████████████████████████████████████████████████▏           | 6/7 [09:02<02:17, 137.44s/it]

In [ ]:
array_of_mean_energies

In [ ]:
data = np.load("matrix_of_mean_energies.txt")
psm = ax.pcolormesh(data, cmap=cmap, rasterized=True, vmin=-4, vmax=4)
ax